In [1]:
import pandas as pd
import numpy as np
import os
import torch
import torch.nn as nn
from dataset import Dataset, to_device
from model import NNModelEx
from time import time
from tqdm.notebook import tqdm
import json
from sklearn.preprocessing import StandardScaler

# Model Saving/Loading Methods

In [2]:
def load_model_only(config):
    path = config.get('model_path', '')
    f = f"{path}/{config['model_identifier']}.pth"
    print("Loading existing model")
    checkpoint = torch.load(f)
    net = checkpoint['net']
    mean_losses = checkpoint['mean_losses']
    return net, mean_losses

def init_h1_weights(m):
    if type(m) == nn.Linear:
        nn.init.kaiming_uniform_(m.weight, mode='fan_in', nonlinearity='relu')
        m.bias.data.fill_(0.01)
        
def load_model_with_config(config, X_train=None, model_width=None, force_train=False):
    # a bit hacky, but in the training phase, we never load and use the minmax scalers
    # just putting it here for when we want to load the model elsewhere THEN revert scaling
    # probably better to have the scalers saved separately....
    device = config.get('device','cpu')
    pyt_device = torch.device(device)
    
    path = config.get('model_path', '')
    f = f"{path}/{config['model_identifier']}.pth"
    
    if os.path.exists(f) and not force_train:
        print("Loading existing model")
        checkpoint = torch.load(f)
        net = checkpoint['net']
        next_epoch = checkpoint['next_epoch']
        loss_func = checkpoint['loss_func']
        optimizer = checkpoint['optimizer']
        scaler = checkpoint['scaler']
        mean_losses = checkpoint['mean_losses']
    else:
        if X_train is None:
            raise Exception('Cannot create model without X_train')
        print("New model created")
        net = NNModelEx(input_size=model_width[0], output_size=model_width[1], **config['model_definition'])
        net.apply(init_h1_weights)
        pos_weight = to_device(torch.tensor([config['loss'].get('pos_weight',1)]), pyt_device) # increase/decrease precision
        loss_func = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
        opt_config=config['optimizer']
        optimizer = torch.optim.SGD(net.parameters(), lr=opt_config['lr'], momentum=opt_config['momentum'])
        scaler = StandardScaler().fit(X_train)
        mean_losses = []
        next_epoch = 0
        save_model_with_config(config, net=net, loss_func=loss_func, optimizer=optimizer,
                               scaler=scaler,
                               mean_losses=mean_losses, next_epoch=next_epoch,
                              )
        # blank scaler when creating new model
    return net, loss_func, optimizer, scaler, mean_losses, next_epoch

def save_model_with_config(config, **kwargs):
    path = config.get('model_path', '')

    f = f"{path}/{config['model_identifier']}.pth"
    torch.save(kwargs, f)

# Model Training/Predicting Methods

In [3]:
def train_model(X_train, y_train, X_test, y_test, configurations, force_train=False):

    path = configurations.get('model_path', None)
    torch.manual_seed(configurations.get('random_seed',0))
    device = configurations.get('device','cpu')
    pyt_device = torch.device(device)

    model_width = (X_train.shape[1], y_train.shape[1])
    net, loss_func, optimizer, scaler, mean_losses, next_epoch, = load_model_with_config(configurations,
                                                                                         X_train,
                                                                                         model_width,
                                                                                         force_train)

    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    
    training_set = Dataset(X_train, y_train)
    training_generator = torch.utils.data.DataLoader(training_set, **configurations['train_params'])
    testing_set = Dataset(X_test, y_test)
    testing_generator = torch.utils.data.DataLoader(testing_set, **configurations['test_params'])
    
    to_device(net, pyt_device)
    net.train()
    print(net)

    if next_epoch == configurations['max_epochs']:
        print("Model finished training. To retrain set force_train = True ")
        net.eval()
        return net, mean_losses

    epbar = tqdm(range(next_epoch, configurations['max_epochs']))
    for epoch in epbar:
        epbar.set_description(f"Epoch {epoch+1}")

        running_eloss = 0
        running_vloss = 0

        ipbar = tqdm(training_generator, leave=False)
        ipbar.set_description(f"Training")

        for i, (x, y) in enumerate(ipbar):
            x = to_device(x, pyt_device)
            y = to_device(y, pyt_device)

            optimizer.zero_grad()
            prediction = net(x)     # input x and predict based on x
            loss = loss_func(prediction, y)     # must be (1. nn output, 2. target)
            loss.backward()         # backpropagation, compute gradients
            optimizer.step()        # apply gradients
            running_eloss += loss.item()

        net.eval()
        mean_vlosses = 0
        predicted_true = 0
        target_true = 0
        correct_true = 0
        
        if configurations['do_validate']:
            with torch.set_grad_enabled(False):
                vpbar = tqdm(testing_generator, leave=False)
                vpbar.set_description("Validating")
                for i, (x, y) in enumerate(vpbar):
                    x = to_device(x, pyt_device)
                    y = to_device(y, pyt_device)
                    prediction = net(x)
                    loss = loss_func(prediction, y)
                    running_vloss += loss.item()
                    
                    # calculate precision https://stackoverflow.com/questions/56643503/efficient-metrics-evaluation-in-pytorch
                    predicted_classes = torch.round(torch.sigmoid(prediction)).squeeze() #.reshape(prediction.shape[0])
                    target_classes = y.squeeze() #.reshape(y.shape[0])
                    target_true += torch.sum(target_classes == 1).float()
                    predicted_true += torch.sum(predicted_classes).float()
                    correct_true += torch.sum((predicted_classes == target_classes) * (target_classes == 1)).float()
                    
            mean_vlosses = running_vloss / len(testing_generator)
            recall = 0 if target_true==0 else float((correct_true / target_true).detach())
            precision = 0 if predicted_true==0 else float((correct_true / predicted_true).detach())
            f1_score = 0 if (precision + recall)==0 else 2 * precision * recall / (precision + recall)

        mean_elosses = running_eloss / len(training_generator)
        mean_losses.append((mean_elosses, mean_vlosses, precision, recall, f1_score))
        save_model_with_config(configurations, net=net, loss_func=loss_func, optimizer=optimizer,
                               scaler=scaler,
                               mean_losses=mean_losses, next_epoch=epoch+1,)
        net.train()

        epbar.set_postfix({'train_loss':f"{mean_elosses:.6f}", 'val_loss':f"{mean_vlosses:.6f}", 'val_prec':f"{precision:.6f}"})
    net.eval()
    
    torch.cuda.empty_cache()
    return net, mean_losses

# Load and Clean Data

In [4]:
def train_test_split(X, y, train_idx=None, test_idx=None):
    X_train = X.loc[train_idx]
    y_train = y.loc[train_idx]
    X_test = X.loc[test_idx]
    y_test = y.loc[test_idx]
    return (X_train, y_train, X_test, y_test)

def load_split_data(suffix=None, split=False):
    if suffix==None:
        suffix='DEFAULT'
        
    X = pd.read_pickle(f'../data/X_{suffix}.pkl')
    y = pd.read_pickle(f'../data/y_{suffix}.pkl')
    
    # Drop NA rows:
    na_rows = X.isna().any(axis=1)
    X = X[~na_rows]
    y = y[~na_rows]
        
    if split:
        X_train, y_train, X_test, y_test = train_test_split(X, y, X.loc['2018':'2020'].index, X.loc['2021':].index)
        return X_train, y_train, X_test, y_test
    else:
        return X, y
    
X_train, y_train, X_test, y_test = load_split_data(suffix='20210801f', split=True)

In [5]:
# Balance data...
add_buys = (y_train.buy==0).sum() - (y_train.buy==1).sum()

y_toadd = y_train[y_train.buy==1].sample(n=add_buys, replace=True, random_state=42)
x_toadd = X_train[y_train.buy==1].sample(n=add_buys, replace=True, random_state=42)

X_train = pd.concat([X_train,x_toadd])
y_train = pd.concat([y_train,y_toadd])

#  Model Creation

In [6]:
model_id = 'alpha13'

config = {
    'model_identifier' : model_id,
    'model_path' : './models',
    'model_type' : 'NNModelEx - Data Variant 20210801f',
    'device' : 'cuda',
    'random_seed' : 0,
    'max_epochs' : 50,
    'do_validate' : True,
    'model_definition' : {
        'layer0' : 1028, 'relu0' : True, 'drop0' : 0.5,
        'layer1' : 1028, 'relu1' : True, 'drop1' : 0.5,
        'layer2' : 1028, 'relu2' : True, 'drop2' : 0.5,
        #'layer3' : 1028, 'relu3' : True, 'drop3' : 0.5,
        #'layer4' : 128, 'relu4' : True, 'drop4' : 0.5,
        #'layer5' : 128, 'relu5' : True, 'drop5' : 0.5,
        #'layer6' : 128, 'relu6' : True, 'drop6' : 0.5,
    },
    'loss' : {
        'pos_weight' : 1
    },
    'optimizer' : {
        'type' : 'torch.optim.SGD',
        'lr' : 0.01,
        'weight_decay' : 0.00025,
        'momentum' : 0.9,
    },
    'train_params' : {
        'batch_size': 200,
        'shuffle': True,
        'num_workers': 5,
        'pin_memory': True,
    },
    'test_params' : {
        'batch_size': 200000,
        'num_workers': 5,
        'pin_memory': True,
    },
}


model_id=config['model_identifier']
with open(f'models/{model_id}.cfg', 'w') as f:
    json.dump(config, f)

# Train

In [7]:
net, mean_losses = train_model(X_train, y_train, X_test, y_test, config, force_train=True)

New model created
NNModelEx(
  (net): Sequential(
    (0): Linear(in_features=542, out_features=1028, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=1028, out_features=1028, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.5, inplace=False)
    (6): Linear(in_features=1028, out_features=1028, bias=True)
    (7): ReLU()
    (8): Dropout(p=0.5, inplace=False)
    (9): Linear(in_features=1028, out_features=1, bias=True)
  )
)


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/774 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/774 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/774 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/774 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
torch.cuda.empty_cache()